In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np
import seaborn as sns 
import pickle

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

In [2]:
df = pd.read_csv('/Users/elmi/Projects/CloudWine/data/raw/winemag-data-130k-v2.csv')
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## TF-IDF Model

In [4]:

vectorizer = TfidfVectorizer(min_df=1, stop_words="english")
X = vectorizer.fit_transform(corpus).toarray()
print(X.shape)

NameError: name 'corpus' is not defined

In [5]:
labels = df['title'].to_numpy()

In [6]:
x = "Clean, cherry-like fruit flavours on the palate, a hint of wood and a touch of bitter almonds, good balance, long, dry finish."

In [19]:
df['description'][0]

"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [7]:
# tf-idf based vectors
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True, max_features = 500000)

# Fit the model
corpus = df['description'].tolist()
tf_transformer = tf.fit(corpus)

# Dump the file
pickle.dump(tf_transformer, open("tfidf1.pkl", "wb"))


# Testing phase
tf1 = pickle.load(open("tfidf1.pkl", 'rb'))

# Create new tfidfVectorizer with old vocabulary
tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True,
                          max_features = 500000, vocabulary = tf1.vocabulary_)

X = tf1_new.fit_transform(corpus)

In [24]:
new_corpus = ["Clean, cherry-like fruit flavours on the palate, a hint of wood and a touch of bitter almonds, good balance, long, dry finish."]
new_corpus = ["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."]
X_tf1 = tf1_new.fit_transform(new_corpus)

In [25]:
cosine_similarity(X_tf1, X) 

array([[0.91858383, 0.00922744, 0.00784255, ..., 0.        , 0.02164481,
        0.00760593]])

In [26]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(X_tf1, X).flatten()

In [27]:
related_docs_indices = cosine_similarities.argsort()[:-5:-1]
related_docs_indices

array([     0, 116562,  36459,   2000])

In [28]:
cosine_similarities[related_docs_indices]

array([0.91858383, 0.23179146, 0.23030275, 0.21852877])

In [35]:
df[df.index.isin(related_docs_indices)]['description']

0         Aromas include tropical fruit, broom, brimston...
2000      The nose isn't very expressive but reveals whi...
36459     This has light floral aromas of dried chamomil...
116562    This opens with aromas of pressed acacia flowe...
Name: description, dtype: object

In [38]:
new_corpus

["Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."]

In [41]:
for i in range(len(related_docs_indices)):
    print(df.iloc[related_docs_indices[i]]['description'])
    print('\n')

Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.


This opens with aromas of pressed acacia flowers, ripe stone fruits and dried sage. The palate isn't overly sweet, offering dried apricot, wildflower honey and toasted almond notes.


This has light floral aromas of dried chamomile and acacia flower. The palate is lean and not very expressive, offering bitter almond and yellow apple skin alongside brisk acidity.


The nose isn't very expressive but reveals white flower and tropical fruit. The simple palate delivers pineapple and lemon zest alongside brisk acidity.




In [37]:
df.iloc[116562]['description']

"This opens with aromas of pressed acacia flowers, ripe stone fruits and dried sage. The palate isn't overly sweet, offering dried apricot, wildflower honey and toasted almond notes."